In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os

In [3]:
def prepro(z):
    prot = pd.DataFrame(columns=['aa','struct'])
    if(z.shape[0]>0):
        current = z['Structure'][0]
        c = 0
        s = ''
        for i in z.index:
            if(z['Structure'][i]==current):
                c = c+1
                s = s+z['AA'][i]
            else:
                if(c==7):
                    prot = prot.append({'aa':s,'struct':current}, ignore_index=True)
                s = z['AA'][i]
                c = 1
                current = z['Structure'][i]
    return prot

In [4]:
#Read Data
df = pd.DataFrame(columns=['aa','struct'])
for infile in os.listdir("~/data/HW3/DSSP"):
    adrs = '~/data/HW3/DSSP/' + infile
    dff = pd.read_csv(adrs,' ')
    dfff = prepro(dff)
    df = df.append(dfff)

In [5]:
# dums = pd.get_dummies(df['struct'])
dums = pd.get_dummies(df['aa'])

In [10]:
data = pd.concat([df,dums],axis=1)

In [11]:
data = data.drop(['aa'],axis=1)

In [12]:
data.head()

,struct,AAAAALA,AAAAKAG,AAAFGNV,AAALRQA,AADLAKL,AAEAEKL,AAEHYKA,AAEIAHT,AAELMQE,...,dAPcTMQ,dFSVQAV,dGVGGSD,eTFGDPI,edEKVNK,hgEKVNK,iGVGGSD,kjEKVNK,mLWLSWK,nFSVQAV
0,H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,T,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#train_test_split
from sklearn.model_selection import train_test_split

In [18]:
y = data['struct']
X = data.drop('struct',axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
rfc = RandomForestClassifier(n_estimators=200)

In [19]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
rfc_pred = rfc.predict(X_test)

In [25]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score

In [28]:
print(confusion_matrix(y_test,rfc_pred))
print('\n')
# print(confusion_matrix(X_test,rfc_pred))
print(accuracy_score(y_test,rfc_pred))
print('\n recall_score:')
print('micro: ',recall_score(y_test,rfc_pred,average='micro'))
print('macro: ',recall_score(y_test,rfc_pred,average='macro'))
print('weighted: ',recall_score(y_test,rfc_pred,average='weighted'))
print('none: ',recall_score(y_test,rfc_pred,average=None))

[[2647    0    0    0    0    0]
 [  18   35    0    0    0    1]
 [ 336    0  971    0    0    0]
 [   6    0    0    4    0    0]
 [  67    0    0    0   47    0]
 [  53    0    0    0    0   83]]


0.8873008434864105

 recall_score:
micro:  0.8873008434864105
macro:  0.6356076152240905
weighted:  0.8873008434864105
none:  [1.         0.64814815 0.74292272 0.4        0.4122807  0.61029412]


In [29]:
from sklearn.neural_network import MLPClassifier

In [30]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
clf.predict_proba(X_test)

array([[7.99053498e-04, 1.52808106e-04, 9.98577207e-01, 7.97510960e-05,
        1.44345862e-04, 2.46834040e-04],
       [9.99535577e-01, 5.37390705e-05, 1.95029219e-04, 7.59938145e-05,
        9.02951315e-05, 4.93661973e-05],
       [1.30357190e-04, 5.54175424e-05, 9.99688531e-01, 2.11920101e-05,
        3.41837904e-05, 7.03181406e-05],
       ...,
       [9.99703303e-01, 4.45745385e-05, 1.06673912e-04, 5.11140356e-05,
        6.58040630e-05, 2.85302079e-05],
       [5.63559816e-05, 3.56656571e-05, 9.99842580e-01, 1.23977767e-05,
        2.24831041e-05, 3.05176031e-05],
       [9.99556593e-01, 5.21958262e-05, 1.63919605e-04, 7.89449439e-05,
        1.00919197e-04, 4.74271064e-05]])

In [31]:
clf_pred = clf.predict(X_test)

In [32]:
clf_score = clf.score(X_test, y_test)

In [33]:
print(confusion_matrix(y_test,clf_pred))
print('\n')
# print(confusion_matrix(X_test,rfc_pred))
print(accuracy_score(y_test,clf_pred))
print('\n recall_score:')
print('micro: ',recall_score(y_test,clf_pred,average='micro'))
print('macro: ',recall_score(y_test,clf_pred,average='macro'))
print('weighted: ',recall_score(y_test,clf_pred,average='weighted'))
print('none: ',recall_score(y_test,clf_pred,average=None))
print('clf_score: ', clf_score)

[[2647    0    0    0    0    0]
 [  18   35    0    0    0    1]
 [ 336    0  971    0    0    0]
 [   6    0    0    4    0    0]
 [  67    0    0    0   47    0]
 [  53    0    0    0    0   83]]


0.8873008434864105

 recall_score:
micro:  0.8873008434864105
macro:  0.6356076152240905
weighted:  0.8873008434864105
none:  [1.         0.64814815 0.74292272 0.4        0.4122807  0.61029412]
clf_score:  0.8873008434864105


In [ ]:
"""
for better results we should set parameters manually, and choose best one.
another point is way coding strings. if order isn't important,
it's better to add at most 2*26 columns instead of ~5000!
"""